# Thanawya ama grades scraping jupyter notebook
### Scrapping `g12.emis.gov.eg` for students grades
![g12.emis.gov.eg](./data/img/screencapture-g12-emis-gov-eg.png)

In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [6]:
students_data = []
url = "https://g12.emis.gov.eg/"
error_msg = 'يرجى مراجعة لجنة النظام والمراقبة التابع لها الطالب' # SeatNo not found
seat_numbers = list(map(str, range(1077439,1897338))) # Inputs => 1002583 ---> 1897338

In [7]:
def getGrades(subject):
    dictionary = {}
    
    for index, subject in enumerate(subject):
        if(index == 0):
            continue
            
        subject_name = subject.th.string.strip()
        subject_grade = subject.th.next_sibling.next_sibling.string.strip()
        dictionary[subject_name] = subject_grade
    
    return dictionary

In [8]:
def getStudentData(soup):
    dictionary = {}    
    student_details = soup.find_all('table')[0].tbody.find_all('tr')
    primary_subjects = soup.find_all('table')[1].find_all('tr')
    secondary_subjects = soup.find_all('table')[2].find_all('tr')
    
    #Student details        
    dictionary['seat_num'] = student_details[0].td.string.strip()
    dictionary['student_name'] = student_details[1].td.string.strip()
    dictionary['school_name'] = student_details[2].td.string.strip()
    for student_detail in student_details[3:]:
        dictionary[student_detail.th.string.strip()] = student_detail.th.next_sibling.next_sibling.string.strip()
    dictionary['status'] = soup.find_all(attrs={'id':"Result"})[0].find('h2').string.strip()
    
    #Student grades
    dictionary.update(getGrades(primary_subjects))
    dictionary.update(getGrades(secondary_subjects))
    
    return dictionary    

In [9]:
try:
    # Get the path to chromedriver
    driver_path = ChromeDriverManager().install()

    # Create a Chrome webdriver
    driver = webdriver.Chrome()

    for seat_number in seat_numbers:
        # Open the URL in the browser
        driver.get(url)

        # Find the input element by its name attribute
        input_element = driver.find_element('id', 'SeatingNo')

        # Enter the seat number in the input field
        input_element.send_keys(seat_number)

        # Submit the form
        input_element.submit()

        # Get the current page source (HTML content) after the form submission
        response_text = driver.page_source
        
        if error_msg in response_text:
            continue
            
        soup = BeautifulSoup(response_text, 'html.parser')
        students_data.append(getStudentData(soup))
        
except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the browser
    driver.quit()

Error: list index out of range


### Convert the `studenst_data` array to a DataFrame

In [20]:
df = pd.DataFrame(students_data)

df.replace(np.nan, 0, inplace=True)
df.sort_values(by='seat_num', ascending=True, axis=0, inplace=True)

df.rename(columns={'اللغة العربية':'arabic', 'التاريخ':'history', 'اللغة الأجنبية الثانية':'second_language', 'اللغة الأجنبية الأولى':'first_language', 'المجموع الكلى':'total',
                  'علم النفس والإجتماع':'psychology_sociology', 'الفلسفة والمنطق':'philosophy_logic', 'الجغرافيا':'geography', 'الفيزياء':'physics',
                   'الكيمياء':'chemistry', 'الجيولوجيا وعلوم البيئة':'geology', 'الأحياء':'biology', 'الرياضيات التطبيقية':'applied_mathematics', 
                  'الرياضيات البحتة':'pure_mathematics', "التربية الدينية":'religion', "التربية الوطنية":'national_education',
                  'الإقتصاد والإحصاء':'Economics_statistics', 'الإحصاء':'statistics'}, inplace=True)


In [31]:
print(df.shape)
subjects = df.columns[9:]
df.columns.to_frame()

(108, 26)


,0
seat_num,seat_num
student_name,student_name
school_name,school_name
المديرية,المديرية
الإدارة,الإدارة
اسم اللجنة,اسم اللجنة
عنوان اللجنة,عنوان اللجنة
الهاتف,الهاتف
status,status
arabic,arabic


### Merging Frames

In [22]:
# df_old = pd.read_csv('./data/thanawya2023_part1.csv')
# df = df_old.append(df)

print(f'Before dropping duplicates {df.shape}')
df = df.drop_duplicates()
print(f'After dropping duplicates {df.shape}')

Before dropping duplicates (108, 26)
After dropping duplicates (108, 26)


In [32]:
# Replacing Subjects values with a translated values
df.loc[:,subjects] = df.loc[:,subjects].replace('غياب','absence')
df.loc[:,subjects] = df.loc[:,subjects].replace('مؤجـل','postponed')
df.loc[:,subjects] = df.loc[:,subjects].replace('معفـي','exempt')
df.loc[:,subjects] = df.loc[:,subjects].replace('إجتاز','passed')
df.loc[:,subjects] = df.loc[:,subjects].replace('ملغـي','cancelled')
df.loc[:,subjects] = df.loc[:,subjects].replace('نصـف','0.5')
df.loc[:,subjects] = df.loc[:,subjects].replace('واحـد','1')
df.loc[:,subjects] = df.loc[:,subjects].replace('صفـر','0')
df.loc[:,subjects] = df.loc[:,subjects].replace(np.nan, 0)

In [33]:
for i in range(4, df.shape[1]):
    print(f' {df.columns[i]} : {df.iloc[:,i].unique()}', "\n"*3)

 الإدارة : ['الشيخ زايد'] 



 اسم اللجنة : ['لجنة النظام والمراقبة قطاع القاهرة أ'] 



 عنوان اللجنة : ['مدرسة السنية الثانوية بنات بالسيدة زينب'] 



 الهاتف : ['0223911294-0223958589'] 



 status : ['ناجـح' 'دور ثان' 'راسـب'] 



 arabic : ['45.5' '42.5' '40' '43' '53.5' '41' '48.5' '54.5' '44' '59.5' '50' '51.5'
 '18.5' '42' '41.5' '47.5' '45' '27.5' '44.5' '50.5' '40.5' '52' '26.5'
 '28' '51' '43.5' 'absence' '48' '29' '47' '55.5' '29.5' '63.5' '23.5'
 '57' '49.5'] 



 first_language : ['26' '12' '25' '35' '27.5' '29.5' '32' '31' '10.5' 'absence' '28.5' '9'
 '3.5' '37.5' '26.5' '11.5' '11' '10' '43'] 



 second_language : ['20' '6' '6.5' '27.5' '32.5' '31.5' '34' '28.5' '22' '35' '30.5' '22.5'
 '25.5' '24.5' '31' '26' '9' '20.5' '21.5' '5' '8' '21' '23.5' 'absence'
 '27' '8.5' '7.5' '9.5' '36.5' '7' '23'] 



 history : ['36' '30' '39' '41' '44' '35' '34.5' '43.5' '41.5' 0 '30.5' 'absence'
 '13' '32.5' '14' '12' '37' '12.5' '10' '38'] 



 geography : ['32' '45' '50' '43' '52'

In [34]:
# Replacing Status values with a translated values
df['status'] = df['status'].replace('راسـب', 'fail')
df['status'] = df['status'].replace('دور ثان', 'second round')
df['status'] = df['status'].replace('ناجـح', 'pass')
df['status'].unique()

array(['pass', 'second round', 'fail'], dtype=object)

In [35]:
df['seat_num'] = df['seat_num'].map(int)
df.sort_values(by='seat_num', ascending=True, axis=0, inplace=True)
df.to_csv('./data/thanawya2023_part2.csv', encoding='utf-8-sig', index=False)

### Viewing csv file

In [42]:
df = pd.read_csv('./data/thanawya2023_part2.csv')
df.shape

(108, 28)

In [43]:
df.head()

,seat_num,student_name,gender,school_name,المديرية,اسم اللجنة,الإدارة,عنوان اللجنة,الهاتف,section,...,psychology_sociology,chemistry,physics,pure_mathematics,applied_mathematics,biology,geology,religion,national_education,Economics_statistics
0,1077439,مهند محمد احمد احمد,M,خدمات ثانوي,الجيزة,لجنة النظام والمراقبة قطاع القاهرة أ,الشيخ زايد,مدرسة السنية الثانوية بنات بالسيدة زينب,0223911294-0223958589,literature,...,32.5,0,0,0,0,0.0,0,20,16,25
1,1077440,نورالدين ايمن السيد محمد,M,خدمات ثانوي,الجيزة,لجنة النظام والمراقبة قطاع القاهرة أ,الشيخ زايد,مدرسة السنية الثانوية بنات بالسيدة زينب,0223911294-0223958589,literature,...,34,0,0,0,0,0.0,0,20,17,13
2,1077441,وليد مصطفي بدري اسماعيل,M,خدمات ثانوي,الجيزة,لجنة النظام والمراقبة قطاع القاهرة أ,الشيخ زايد,مدرسة السنية الثانوية بنات بالسيدة زينب,0223911294-0223958589,literature,...,35,0,0,0,0,0.0,0,20,17,25
3,1077442,يوسف احمد محمد محمود,M,خدمات ثانوي,الجيزة,لجنة النظام والمراقبة قطاع القاهرة أ,الشيخ زايد,مدرسة السنية الثانوية بنات بالسيدة زينب,0223911294-0223958589,literature,...,31,0,0,0,0,0.0,0,20,16,25
4,1077443,يوسف خالد محمد ابوالحسن,M,خدمات ثانوي,الجيزة,لجنة النظام والمراقبة قطاع القاهرة أ,الشيخ زايد,مدرسة السنية الثانوية بنات بالسيدة زينب,0223911294-0223958589,literature,...,38,0,0,0,0,0.0,0,21,21,25


In [44]:
df.tail()

,seat_num,student_name,gender,school_name,المديرية,اسم اللجنة,الإدارة,عنوان اللجنة,الهاتف,section,...,psychology_sociology,chemistry,physics,pure_mathematics,applied_mathematics,biology,geology,religion,national_education,Economics_statistics
103,1077544,ملك عادل سيد ثابت,F,خدمات الشيخ زايد,الجيزة,لجنة النظام والمراقبة قطاع القاهرة أ,الشيخ زايد,مدرسة السنية الثانوية بنات بالسيدة زينب,0223911294-0223958589,literature,...,35,0,0,0,0,0.0,0,13,16,12
104,1077545,منه وليد درويش محمد,F,خدمات الشيخ زايد,الجيزة,لجنة النظام والمراقبة قطاع القاهرة أ,الشيخ زايد,مدرسة السنية الثانوية بنات بالسيدة زينب,0223911294-0223958589,literature,...,30,0,0,0,0,0.0,0,10,13,25
105,1077546,مي سيد حسين سيد,F,خدمات الشيخ زايد,الجيزة,لجنة النظام والمراقبة قطاع القاهرة أ,الشيخ زايد,مدرسة السنية الثانوية بنات بالسيدة زينب,0223911294-0223958589,literature,...,42.5,0,0,0,0,0.0,0,15,14,25
106,1077547,ميار ابراهيم رفيق امين,F,خدمات الشيخ زايد,الجيزة,لجنة النظام والمراقبة قطاع القاهرة أ,الشيخ زايد,مدرسة السنية الثانوية بنات بالسيدة زينب,0223911294-0223958589,literature,...,35,0,0,0,0,0.0,0,13,15,25
107,1077548,نانسي حسام رمضان محمد خليل,F,خدمات الشيخ زايد,الجيزة,لجنة النظام والمراقبة قطاع القاهرة أ,الشيخ زايد,مدرسة السنية الثانوية بنات بالسيدة زينب,0223911294-0223958589,literature,...,34,0,0,0,0,0.0,0,13,17,25
